<a href="https://colab.research.google.com/github/jasial2/JapaneseTranscription/blob/main/DWT_Transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Added 'onnxruntime-gpu' to fix the VAD warning and speed up processing
!pip install whisper-timestamped srt ffmpeg-python tqdm onnxruntime-gpu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 54.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.5 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=9fba121bc011e4d7d4fdd9e9d29bb6851547f244594c30105322533cbfd900be
  Stored in directory: /root/.cache/pip/wheels/7e/75/5b/e1d5c3756631e4bda806f6cc9640153b39484bb6f7b0b8def3
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=80397

In [ ]:
# --- STEP 2: SETUP & IMPORTS ---
import whisper_timestamped as whisper
import torch
import os
import srt
import datetime
import ffmpeg
import re
import sys
import warnings
from tqdm import tqdm

# --- FIX: SUPPRESS TORCH HUB WARNINGS ---
warnings.filterwarnings("ignore", category=UserWarning, module="torch.hub")

# --- MISSING PART: DEFINE DEVICE ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.

Using device: cuda


In [ ]:
# Load Model
# Note: 'large-v3' is usually better for timestamps than 'turbo', but 'turbo' is faster.
print("Loading model...")
model = whisper.load_model("turbo", device=device)

Loading model...


100%|██████████████████████████████████████| 1.51G/1.51G [00:11<00:00, 145MiB/s]


In [ ]:
# --- CONFIGURATION ---
input_video = "1234.mp3"  # <--- REPLACE THIS WITH YOUR FILE
output_srt = "output.srt"
language = "ja"

# --- HELPER: AUDIO PROCESSING ---
processed_audio = "temp_clean_audio.wav"

if not os.path.exists(input_video):
    raise FileNotFoundError(f"File '{input_video}' not found! Upload it to the Files tab.")

print(f"1. Optimizing audio for Voice Frequencies ({input_video})...")
try:
    # highpass=100: Removes rumble. lowpass=8000: Removes hiss.
    # ar=16000: Whisper requires 16kHz.
    (
        ffmpeg.input(input_video)
        .output(processed_audio, acodec="pcm_s16le", ac=1, ar="16000", af="highpass=f=100,lowpass=f=8000")
        .overwrite_output()
        .run(quiet=True)
    )
except ffmpeg.Error as e:
    print("FFmpeg error:", e.stderr)
    raise

# --- 2. RUN WHISPER-TIMESTAMPED (The Sync Fix) ---
print("2. Transcribing with Forced Alignment (DTW)...")

# Configuration for Sync Accuracy
result = whisper.transcribe(
    model,
    processed_audio,
    language=language,

    # --- SYNC SETTINGS ---
    beam_size=5,
    best_of=5,
    temperature=0.0,

    # CRITICAL: Forces alignment to audio waves, fixing the "out of sync" issue
    trust_whisper_timestamps=False,

    # Helps ignore silence/breathing so timestamps don't drift
    vad=False,

    # Detects hesitancy (uh, um) separately (optional, helps precision)
    detect_disfluencies=True,

    # Standard settings
    condition_on_previous_text=False,
    initial_prompt="うめき声や呼吸音を無視して、会話のみを書き起こしてください。"
)

# --- 3. ADVANCED FILTERING ---
print("3. Applying Japanese Garbage & Duration Filters...")

# A. Hallucination Triggers
hallucination_triggers = [
    "thank you for watching", "thanks for watching", "please subscribe",
    "subscribe", "sub by", "translated by", "amara", "viewing",
    "see you next", "bye", "the end", "like and", "follow me",
    "字幕", "視聴", "チャンネル", "登録", "高評価"
]

# B. Garbage Sounds
garbage_exact_matches = {
    # English
    "a", "aa", "ah", "ahh", "ha", "haa", "hah", "haha",
    "mm", "mmm", "hmm", "mh", "oh", "huh", "o", "m",
    "h", "eh", "uh", "uhh",
    # Japanese
    "あ", "ああ", "あっ", "あー",
    "ん", "んん", "んっ", "う", "うっ",
    "は", "はぁ", "はあ", "ふ", "ふぅ",
    "く", "くっ", "や", "いや", "お", "おっ"
}

final_subs = []
sub_index = 1

for segment in result["segments"]:
    text = segment["text"].strip()
    text_lower = text.lower()

    # 1. DURATION CHECK
    # Check strict duration to remove noise
    duration = segment["end"] - segment["start"]
    if duration < 0.4:
        continue

    # 2. HALLUCINATION CHECK
    if any(h in text_lower for h in hallucination_triggers):
        continue

    # 3. GARBAGE CHECK
    clean_text = re.sub(r'[^\w\s]', '', text_lower) # Remove punctuation
    words = clean_text.split()

    if len(words) == 0:
        continue

    # Check if ALL words are garbage
    is_pure_garbage = True
    for w in words:
        if w not in garbage_exact_matches:
            is_pure_garbage = False
            break

    if is_pure_garbage:
        continue

    # 4. REPETITION CHECK
    if len(words) > 4 and len(set(words)) == 1:
        continue

    # Add to subtitles
    final_subs.append(
        srt.Subtitle(
            index=sub_index,
            start=datetime.timedelta(seconds=segment["start"]),
            end=datetime.timedelta(seconds=segment["end"]),
            content=text
        )
    )
    sub_index += 1

# --- 4. SAVE FILE ---
with open(output_srt, "w", encoding="utf-8") as f:
    f.write(srt.compose(final_subs))

# Cleanup
if os.path.exists(processed_audio):
    os.remove(processed_audio)

print(f"------------------------------------------------")
print(f"✅ Optimization Complete.")
print(f"Saved: {output_srt}")

1. Optimizing audio for Voice Frequencies (1234.mp3)...
2. Transcribing with Forced Alignment (DTW)...


100%|██████████| 187317/187317 [02:52<00:00, 1088.41frames/s]


3. Applying Japanese Garbage & Duration Filters...
------------------------------------------------
✅ Optimization Complete.
Saved: output.srt
